In [ ]:
import os
import numpy as np
import nibabel
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm


In [ ]:
flair_files={}
t1_files={}
t2_files={}
t1ce_files={}
seg_files={}

file_img_root="Preprocessed_BraTS_Data"

#creating list of all files
for root, dirs, files in os.walk(file_img_root):
    for name in files:
        indx = name.find('_')
        key = name[indx+1:indx+6]
        file_path = os.path.join(root, name)
            
        if 'flair.nii.gz' in name:
            flair_files[key] = file_path
        elif 't1.nii.gz' in name:
            t1_files[key] = file_path
        elif 't1ce.nii.gz' in name:
            t1ce_files[key] = file_path
        elif 't2.nii.gz' in name:
            t2_files[key] = file_path
        elif 'seg.nii.gz' in name:
            seg_files[key] = file_path


In [ ]:
#check that every segmentation file (ground truth) has corresponding X data
for k in seg_files.keys():
    if k not in flair_files.keys():
        print('Not found in flair:', k)
    if k not in t1_files.keys():
        print('Not found in t1:', k)
    if k not in t1ce_files.keys():
        print('Not found in t1ce:', k)
    if k not in t2_files.keys():
        print('Not found in t2:', k)

In [ ]:
# didn't find all files for sample 01627
del seg_files['01627']
len(seg_files)

In [ ]:
def func_resize_depth_and_width(img, new_dims = [120,120]):
    resized_img = np.zeros((img.shape[0],new_dims[0], new_dims[1], img.shape[3]))

    for i in range(img.shape[0]):
        resized_img[i,] = tf.image.resize(img[i], [120,120])
    
    return resized_img

def func_resize_height(img, new_height=80):
    resized_img = np.zeros((80,img.shape[1],img.shape[2],img.shape[3]))
    
    for i in range(img.shape[1]):
        resized_img[:,i,:,:] = tf.image.resize(img[:,i,:,:], [80,120])

    return resized_img

In [ ]:
save_root = 'Resized_MRI_Volumes'
file_prefix = 'resized_sample_'

for sample in tqdm(seg_files.keys()):
    img_affine = nibabel.load(flair_files[sample]).affine
    
    img = np.stack((nibabel.load(flair_files[sample]).get_fdata(), 
                nibabel.load(t1_files[sample]).get_fdata(), 
                nibabel.load(t1ce_files[sample]).get_fdata(), 
                nibabel.load(t2_files[sample]).get_fdata()),
               axis = -1)
    img_resized_dw = func_resize_depth_and_width(img, new_dims=[120,120])
    img_resized_all_dims = func_resize_height(img_resized_dw, new_height=80)
    
    assert(img_resized_all_dims.shape == (80,120,120,4))
    
    file_name = file_prefix + sample + '.nii'
    new_img = nibabel.Nifti1Image(img_resized_all_dims, img_affine)
    nibabel.save(new_img, os.path.join(save_root,file_name))


In [ ]:
def seg_resize_dw(img, new_dims = [120,120]):
    resized_img = np.zeros((img.shape[0],new_dims[0], new_dims[1], img.shape[3]))

    for i in range(img.shape[0]):
        resized_img[i,] = tf.image.resize(img[i], [120,120])
    
    return resized_img

def seg_resize_h(img, new_height=80):
    resized_img = np.zeros((80,img.shape[1],img.shape[2], img.shape[3]))
    
    for i in range(img.shape[1]):
        resized_img[:,i,:] = tf.image.resize(img[:,i,:], [80,120])

    return resized_img
    

In [ ]:
#resizing segmentation files and using binary labels {0,1}

save_root = 'Resized_seg_files'
file_prefix = 'resized_'

for k, v in tqdm(seg_files.items()):
    img = nibabel.load(v)
    img_affine = img.affine
    img_data = img.get_fdata()
    
    img_data = np.expand_dims(img_data, axis=-1)
    
    img_data = seg_resize_dw(img_data, new_dims=[120,120])
    img_data = seg_resize_h(img_data, new_height=80)
    img_data = (img_data > 0).astype('uint8')
    
    assert(img_data.shape == (80,120,120,1))
    
    file_name = file_prefix + k + '.nii.gz'
    new_img = nibabel.Nifti1Image(img_data, img_affine)
    nibabel.save(new_img, os.path.join(save_root,file_name))
